In [1]:
# Load in our libraries
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')

# Going to use these 5 base models for the stacking
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

PassengerId = test["PassengerId"]

train.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [3]:
train["Ticket_type"] = train["Ticket"].apply(lambda x: x[0:3])
train["Ticket_type"] = train["Ticket_type"].astype("category")
train["Ticket_type"] = train["Ticket_type"].cat.codes

test["Ticket_type"] = test["Ticket"].apply(lambda x: x[0:3])
test["Ticket_type"] = test["Ticket_type"].astype("category")
test["Ticket_type"] = test["Ticket_type"].cat.codes

train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Ticket_type
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,124
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,137
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,148
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,3
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,97


In [4]:
full_data = [train, test]

train["Words_Count"] = train["Name"].apply(lambda x: len(x.split()))
test["Words_Count"] = test["Name"].apply(lambda x: len(x.split()))

In [6]:
train["Has_Cabin"] = train["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
test["Has_Cabin"] = test["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

In [40]:
x = train["Age"][np.isnan(train["Age"])]
x = 2

In [ ]:
#SibSpとParchと自身を合体させてFamilySizeという特徴を作る
for dataset in full_data:
    dataset["FamilySize"] = dataset["SibSp"] + dataset["Parch"] + 1

#FamilySize=1のIsAloneという特徴を作る
for dataset in full_data:
    dataset["IsAlone"] = 0
    dataset.loc[dataset["FamilySize"] == 1, "IsAlone"] = 1
    
for dataset in full_data:
    dataset["Embarked"] = dataset["Embarked"].fillna("S")
    
for dataset in full_data:
    dataset["Fare"] = dataset["Fare"].fillna(train["Fare"].median())

train["CategoricalFare"] = pd.qcut(train["Fare"], 4)

for dataset in full_data:
    age_avg = dataset["Age"].mean()
    age_std = dataset["Age"].std()
    age_null_count = dataset["Age"].isnull().sum()
    age_null_random_list = np.random.randint(ave_avg - age_std, age_avg + age_std, size=age_null_count)
    dataset["Age"][np.isnan(dataset["Age"])] = age_null_random_list
    dataset["Age"] = dataset["Age"].astype(int)

train["CategoricalAge"] = pd.qcut(train["Age"], 5)

def get_title(name):
    isTitle = re.search(" ([A-Za-z]+)\.", name)
    if isTitle:
        return isTitle.group(1)
    return ""

for dataset in full_data:
    dataset["Title"] = dataset["Name"].apply(get_title)
    
for dataset in full_data:
    dataset["Title"] = dataset["Title"].replace(["Lady", "Countess", "Capt", "Col", "Don", "Dr", "Major", "Rev", 
                                                "Sir", "Jonkheer", "Dona"], "Rare")
    dataset["Title"] = dataset["Title"].replace("Mlle", "Miss")
    dataset["Title"] = dataset["Title"].replace("Ms", "Miss")
    dataset["Title"] = dataset["Title"].replace("Mme", "Mrs")